In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import random
from urllib.parse import urljoin  # ✅ para armar bien los links

In [ ]:
# Diccionario con ciudad y su código en el parámetro x=
CITY_CODES = {"Montevideo": 1,
              "Canelones": 3,
              "Maldonado": 10}

BASE_URL = "https://casasweb.com/resultados.aspx?a=Alquiler_Apartamento&z=0&x={}&m=&n=A&t=a&d=x&b=x&i=0&h=0&c=1"

In [ ]:
# --- Helpers ---
def parse_number(text):
    """Convierte texto como 'USD 195' o '79 m²' en int."""
    if not text:
        return None
    m = re.search(r'(\d[\d\.,]*)', text)
    if not m:
        return None
    num = m.group(1).replace(".", "").replace(",", "")
    try:
        return int(num)
    except:
        return None

In [ ]:
def scrap_city(city_name, city_code):
    url = BASE_URL.format(city_code)
    print(f"📍 Scrapeando {city_name} -> {url}")

    propiedades = []

    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=15)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        anuncios = soup.find_all("div", class_="card mb-4 border-0")

        for anuncio in anuncios:
            if len(propiedades) >= 10:  # mínimo 10 propiedades por ciudad
                break
            try:
                # Link (ahora con urljoin)
                a = anuncio.find("a", href=True)
                link = urljoin("https://casasweb.com", a["href"]) if a else None

                # Precio (<h2 class="my-0">)
                precio = None
                h2 = anuncio.find("h2", class_="my-0")
                if h2:
                    precio = parse_number(h2.get_text(" ", strip=True))

                # HABITACIONES (buscar en item-det los <b> que digan "Dorm")
                habitaciones = None
                item_det = anuncio.find("div", class_="card-text small mb-2 text-center item-det")
                if item_det:
                    for b in item_det.find_all("b"):
                        texto_b = b.get_text(strip=True)
                        if "dorm" in texto_b.lower():
                            habitaciones = parse_number(texto_b)
                            break

                # TAMAÑO (<i>)
                tamano = None
                i_tag = anuncio.find("i")
                if i_tag:
                    tamano = parse_number(i_tag.get_text(strip=True))

                # Guardar propiedad
                propiedades.append({"precio": precio,
                    "tamano": tamano,
                    "habitaciones": habitaciones,
                    "link": link
                })

                print(f"   ✔ {city_name}: precio={precio}, tam={tamano}, hab={habitaciones}, link={link}")

                time.sleep(random.uniform(0.2, 0.4))  # pequeña pausa
            except Exception as e:
                print(f"⚠️ Error en anuncio: {e}")
                continue
    except Exception as e:
        print(f"❌ Error general en {city_name}: {e}")

    return {"nombre": city_name, "propiedades": propiedades}

In [1]:

def main():
    resultado = {"ciudades": []}

    for city, code in CITY_CODES.items():
        ciudad_data = scrap_city(city, code)
        resultado["ciudades"].append(ciudad_data)

    # Guardar en JSON
    with open("propiedades.json", "w", encoding="utf-8") as f:
        json.dump(resultado, f, ensure_ascii=False, indent=2)

    print("\n✅ Datos guardados en propiedades.json")

if __name__ == "__main__":
    main()

📍 Scrapeando Montevideo -> https://casasweb.com/resultados.aspx?a=Alquiler_Apartamento&z=0&x=1&m=&n=A&t=a&d=x&b=x&i=0&h=0&c=1
   ✔ Montevideo: precio=1100, tam=70, hab=3, link=https://casasweb.com/ALQUILER__INNOVA_PROPIEDADES_APARTAMENTO_POCITOS_MONTEVIDEO_CW223700
   ✔ Montevideo: precio=3000, tam=140, hab=3, link=https://casasweb.com/ALQUILER_VENTA_PÉREZ_DEL_CASTILLO_APARTAMENTO_CARRASCO_MONTEVIDEO_CW230762
   ✔ Montevideo: precio=3200, tam=198, hab=3, link=https://casasweb.com/ALQUILER__XOHA_BIENES_RAÍCES_APARTAMENTO_CARRASCO_MONTEVIDEO_CW236747
   ✔ Montevideo: precio=3500, tam=171, hab=4, link=https://casasweb.com/ALQUILER__SAROKA_PROPIEDADES_APARTAMENTO_CARRASCO_MONTEVIDEO_CW240862
   ✔ Montevideo: precio=8300, tam=72, hab=3, link=https://casasweb.com/ALQUILER__ARTIGAS_INMOBILIARIA_APARTAMENTO_AGUADA_MONTEVIDEO_CW148365
   ✔ Montevideo: precio=8500, tam=17, hab=None, link=https://casasweb.com/ALQUILER_VENTA_ASTRO_BIENES_RAÍCES_APARTAMENTO_CIUDAD_VIEJA_MONTEVIDEO_CW234176
   ✔ Mon